In [1]:
import pennylane as qml
from pennylane import numpy as np

np.random.seed(42)

## Constants
Below are constants defined for the code

In [2]:
NUM_QUBITS = 4
RABI_FREQUENCY = 50 * (10 ** 3)
OMEGA_M = 2.75 * (10 ** 6)
MU = 10 * (10 ** 3)
NUM_BLOCKS = 3
PHI_PRECESION = 21
PHI_RANGE = np.linspace(-np.pi, np.pi, PHI_PRECESION)

device = qml.device('default.qubit', wires=NUM_QUBITS)

## Circuit for Quantum Sensing

### Entanglement layer

$$
H_s \;=\; \sum_{i<j} J_{ij}\,\sigma_i^{\phi}\sigma_j^{\phi}
$$

$$
J_{ij} \;=\; \Omega_i \Omega_j \sum_{m} \eta_{i m}\,\eta_{j m}\,
\frac{\mu}{\mu^{2}-\omega_m^{2}}
$$

$\sigma^{\phi}_i = \cos(\phi)\sigma^x_i - \sin(\phi)\sigma^y_i$

For now assuming
- $\eta_{i m} = 0.1 * (-1)^i$
- $\Omega_{i,j} = 50 Khz$
- $\mu = 10 KHz\ to\ 20 KHz$
- $\omega_{m} = 2.75 Mhz$
- $\phi$ = trainable parameter


In [3]:
def J(i, j):
    n_im = 0.1 * ((-1)**i)
    n_jm = 0.1 * ((-1)**j)
    return RABI_FREQUENCY * RABI_FREQUENCY * n_im * n_jm * MU / (MU**2 - OMEGA_M**2)

def sigma_phi_tensor(phi, i, j):
    """Properly define σ_phi(i) ⊗ σ_phi(j)"""
    return (
        (qml.math.cos(phi)**2) * qml.prod(qml.PauliX(i), qml.PauliX(j))
        + (qml.math.sin(phi)**2) * qml.prod(qml.PauliY(i), qml.PauliY(j))
        + (qml.math.cos(phi) * qml.math.sin(phi)) *
          (qml.prod(qml.PauliX(i), qml.PauliY(j)) + qml.prod(qml.PauliY(i), qml.PauliX(j)))
    )

# def entanglement_layer(phi):
#     """Build an exponential of a Hamiltonian with sigma_phi tensor products"""
#     coeffs = []
#     observables = []
#     for i in range(NUM_QUBITS):
#         for j in range(i + 1, NUM_QUBITS):
#             coeffs.append(J(i, j))
#             observables.append(sigma_phi_tensor(phi, i, j))
# 
#     qml.exp(qml.dot(coeffs, observables), coeff=-1j)


def entanglement_layer(phi):
    """Build an exponential of a Hamiltonian with sigma_phi tensor products"""
    sum_op = 0.0 * qml.PauliZ(1) # Creating a zero operator
    for i in range(NUM_QUBITS):
        for j in range(i + 1, NUM_QUBITS):
            sum_op += sigma_phi_tensor(phi, i, j) * J(i, j)

    qml.exp(sum_op, coeff=-1j)



In [4]:
# Testing the entanglement layer working

@qml.qnode(device)
def entanglement_circuit(phi):
    entanglement_layer(phi)
    return qml.probs(wires=range(NUM_QUBITS))


drawer = qml.draw(entanglement_circuit, show_all_wires=True)
print(drawer(np.pi/4))
print(sigma_phi_tensor(np.pi/4, 1, 2))

0: ─╭Exp(-0.00-1.00j 𝓗)─┤ ╭Probs
1: ─├Exp(-0.00-1.00j 𝓗)─┤ ├Probs
2: ─├Exp(-0.00-1.00j 𝓗)─┤ ├Probs
3: ─╰Exp(-0.00-1.00j 𝓗)─┤ ╰Probs
0.5000000000000001 * (X(1) @ X(2)) + 0.4999999999999999 * (Y(1) @ Y(2)) + 0.5 * (X(1) @ Y(2) + Y(1) @ X(2))


### Encoder and Decoder circuit
In Encoder circuit the order of operations are
1. Single qubit parameterized Rx's rotations
2. Interaction from exponentiated hamiltonian of Zig Zag

For decoder it is the same but in opposite order

In [5]:
def encoder_circuit(theta):
    single_rotation_param, entanglement_rotation_param = theta
    
    # Single qubit rotations layer
    for qubit in range(NUM_QUBITS):
        qml.RX(single_rotation_param, wires=qubit)
        
    # Entanglement layer    
    entanglement_layer(entanglement_rotation_param)

def decoder_circuit(nu):
    single_rotation_param, entanglement_rotation_param = nu 
    
    # Entanglement layer    
    entanglement_layer(entanglement_rotation_param)
    
    # Single qubit rotations layer
    for qubit in range(NUM_QUBITS):
        qml.RX(single_rotation_param, wires=qubit)


### Sensor circuit
It consists of
1. Sequence of RY gates
2. Parameterized encoder layer repeated NUM_BLOCKS times.
3. Sensing layer
4. Parameterized decoder layer repeated NUM_BLOCK times.
5. Sequence of RX gates

In [6]:
@qml.qnode(device)
def sensor_circuit(phi: np.complex128, theta, nu):
    for qubit in range(NUM_QUBITS):
        qml.RY(np.pi/2, wires = qubit)

    # Encoder blocks 
    for t in theta:
        encoder_circuit(t)

    for qubit in range(NUM_QUBITS):
        qml.exp(qml.PauliZ(qubit), coeff= -1j * phi)

    # Decoder circuit
    for n in nu:
        decoder_circuit(n)

    for qubit in range(NUM_QUBITS):
        qml.RX(np.pi/2, wires = qubit)

    return qml.probs(wires=range(NUM_QUBITS))

In [7]:
# Drawing Sensor circuit
theta = np.random.uniform(-np.pi, np.pi, (NUM_BLOCKS, 2))
nu = np.random.uniform(-np.pi, np.pi, (NUM_BLOCKS, 2))

drawer = qml.draw(sensor_circuit, show_all_wires=False)
print(drawer(1.0, theta, nu))

0: ──RY(1.57)──RX(-1.99)─╭Exp(-0.00-1.00j 𝓗)──RX(0.61)─╭Exp(-0.00-1.00j 𝓗)──RX(-2.51) ···
1: ──RY(1.57)──RX(-1.99)─├Exp(-0.00-1.00j 𝓗)──RX(0.61)─├Exp(-0.00-1.00j 𝓗)──RX(-2.51) ···
2: ──RY(1.57)──RX(-1.99)─├Exp(-0.00-1.00j 𝓗)──RX(0.61)─├Exp(-0.00-1.00j 𝓗)──RX(-2.51) ···
3: ──RY(1.57)──RX(-1.99)─╰Exp(-0.00-1.00j 𝓗)──RX(0.61)─╰Exp(-0.00-1.00j 𝓗)──RX(-2.51) ···

0: ··· ─╭Exp(-0.00-1.00j 𝓗)──Exp(0.00-1.00j Z)─╭Exp(-0.00-1.00j 𝓗)──RX(-1.04) ···
1: ··· ─├Exp(-0.00-1.00j 𝓗)──Exp(0.00-1.00j Z)─├Exp(-0.00-1.00j 𝓗)──RX(-1.04) ···
2: ··· ─├Exp(-0.00-1.00j 𝓗)──Exp(0.00-1.00j Z)─├Exp(-0.00-1.00j 𝓗)──RX(-1.04) ···
3: ··· ─╰Exp(-0.00-1.00j 𝓗)──Exp(0.00-1.00j Z)─╰Exp(-0.00-1.00j 𝓗)──RX(-1.04) ···

0: ··· ─╭Exp(-0.00-1.00j 𝓗)──RX(0.95)─╭Exp(-0.00-1.00j 𝓗)──RX(1.39)──RX(1.57)─┤ ╭Probs
1: ··· ─├Exp(-0.00-1.00j 𝓗)──RX(0.95)─├Exp(-0.00-1.00j 𝓗)──RX(1.39)──RX(1.57)─┤ ├Probs
2: ··· ─├Exp(-0.00-1.00j 𝓗)──RX(0.95)─├Exp(-0.00-1.00j 𝓗)──RX(1.39)──RX(1.57)─┤ ├Probs
3: ··· ─╰Exp(-0.00-1.00j 𝓗)──RX(0.95)─╰Exp(-0.00-

In [8]:
# Trial running the circuit to check working 
probabilities = sensor_circuit(np.pi/4, theta, theta)
probabilities

tensor([4.78398460e-05, 8.62270463e-04, 8.62270463e-04, 7.05661702e-03,
        8.62270463e-04, 2.89596535e-02, 7.05661702e-03, 9.77749401e-02,
        8.62270463e-04, 7.05661702e-03, 2.89596535e-02, 9.77749401e-02,
        7.05661702e-03, 9.77749401e-02, 9.77749401e-02, 5.19257543e-01], requires_grad=True)

# Cost Calculations

The cost function to optimize is 

$C(\theta, \nu, a) = \int d\phi MSE(\phi) P_{\partial\phi}(\phi)$

where 

- $MSE(\phi) = \Sigma_m [\phi - \phi_{est}(m)]^2 p_{\theta, \nu}(m | \phi)$
- $\phi_{est} = a*m$, "a" is a trainable parameter and "m" is the measured value
- $ p_{\theta, \nu}(m | \phi) $ is probability of measuring state m given $\theta$ and $\nu$
- $P_{\partial\phi}(\phi)$ = ??
- "m" is sum of all spins and can range from `-NUM_QUBITS` ( all spin up) to `+NUM_QUBITS` (all spin up)


In [9]:
def state_to_magnetization(state : int):
   binary_representation = bin(state)
   hamming_weight = binary_representation.count('1')
   
   # When all spin down, hamming weight = N => m = -N
   # When all spin up,   hamming weight = 0 => m = N 
   return NUM_QUBITS - 2 * hamming_weight

def prior_wrapped_gaussian(phi, delta=0.3, k_max=5):
    ''' 
    Phi is periodic, so we make the gaussian distribution periodic by shifting and summing it.
    :param delta is the variance of distribution
    :kmax is the number of shifts of gaussian distribution added. 
    '''
    return sum(
        np.exp(-((phi + 2*np.pi*k)**2) / (2 * delta**2))
        for k in range(-k_max, k_max+1)
    ) / (np.sqrt(2 * np.pi) * delta)

def mean_square_error(phi, theta, nu, a):
    mse = 0.0
    probabilities = sensor_circuit(phi, theta, nu)
    for state, prob in enumerate(probabilities):
        magnetization = state_to_magnetization(state)
        phi_est = a * magnetization
        mse += ((phi_est - phi)**2) * prob
        
    return mse


def cost_function(theta, nu, a):
    cost = 0.0
    for phi in PHI_RANGE:
        cost += mean_square_error(phi, theta, nu, a) * prior_wrapped_gaussian(phi)
    
    return qml.math.toarray(cost)

In [10]:
# Checking if cost is working or not
theta = np.random.uniform(-np.pi, np.pi, (NUM_BLOCKS, 2))
nu = np.random.uniform(-np.pi, np.pi, (NUM_BLOCKS, 2))
cost_function(theta, theta, 0.56)

array(4.47512383)

# Optimization loop

In [11]:
num_steps = 50

# Mark parameters as trainable
theta = np.random.uniform(-np.pi, np.pi,(NUM_BLOCKS, 2), requires_grad=True)
nu = np.random.uniform(-np.pi, np.pi, (NUM_BLOCKS, 2), requires_grad=True)
a = np.random.uniform(-np.pi, np.pi, requires_grad=True)

cost_history = []

optimizer = qml.GradientDescentOptimizer(0.01)
for step in range(num_steps):
    params, cost = optimizer.step_and_cost(cost_function, theta, nu, a)
    cost_history.append(cost)
    if step % 5 == 0:
        print(f"Step: {step}, Cost: {cost}")

TypeError: Can't differentiate w.r.t. type <class 'pennylane.ops.op_math.sprod.SProd'>